# Tableau de bord à destination des Pouvoirs Organisateurs

## 1. Demande
Copie du mail de Chantal Dassonville du 28/10/2021 : 
> _Pour me permettre de transférer les fichiers aux FPO, peux-tu regrouper les feuilles PO, Implantations et Bâtiments par type de PO ?
Donc un fichier pour WBE, un pour libre confessionnel, un pour public subventionné, ….
L’idéal serait de pouvoir transmettre ces fichiers aux FPO juste après le congé. Possible pour toi ?_

## 1.1. Corrections à apporter
Copie du mail de Chantal Dassonville du 08/11/2021 :
>_Pour la bonne lisibilité des tableaux, je propose d’ajouter en tête à côté du nombre d’observations, le nombre total de la population avec le % que cela représente. Il s’agit de voir rapidement pour le lecteur quelle est la part de répondants de son réseau par apport à la part de non répondants.
Pour les PO c’est à la page 10 du rapport sonecom (figure 1)
Pour les implantations c’est page 15 figure 9. Là j’ai un petit souci. Exemple pour le libre confessionnel tu as 1215 et la figure 9 donne 1289 à quid de la différence ? WBE c’est 796 chez toi et 873 cher Sonecom ?
Il y a un truc ?
Pour les bâtiments nous ne connaissons pas la population de référence._
>_La Cocof est un cas particulier à il faut la mettre dans provincial et renommer le fichier en ajoutant provincial_Cocof_

Modifications à réaliser :
- nombre total de la population et proportion
- intégrer les résultats COCOF dans le fichier "Subventionné provincial"

## 2. Livrables
Un tableau de bord (fréquences simples et brutes) des résultats de l'enquête (trois questionnaire) ventilé par réseau d'enseignement. Donc, pour chaque réseau, un fichier Excel à l'intérieur duquel on retrouvera les résultats aux trois questionnaires, un onglet par questionnaire.

## 3. Fichiers source
Les résultats de l'enquête ont été transmis par la SONECOM au travers de trois fichiers Excel. Pour le questionnaire Bâtiments, il y a deux fichiers. On conservera l'ancien fichier car c'est dans celui-ci que se trouvent les informations relatives au questionnaire.

In [2]:
folder = r"C:\dev2122\data\SONECOM"
rapport = r"C:\dev2122\data\SONECOM\rapports_complémentaires\Analyse par PO"
po_file = r"\Export_BD complète_PO.xlsx"
bat_file_old = r"\Export_BD complète_Batiment.xlsx" # contient informations
bat_file = r"\CLEF-WB - Export et TB - BD Batiment vérifiée.xlsx" # chiffres corrigées
impl_file = r"\Export_BD complète _Implantation.xlsx"

## 4. Analyse Exploratoire des données

In [3]:
import pandas as pd
import xlsxwriter
from xlsxwriter import Workbook
import numpy as np
import os
p_df = pd.read_excel(folder+po_file)
b_df = pd.read_excel(folder+bat_file)
i_df = pd.read_excel(folder+impl_file)
dfs = (p_df, b_df, i_df)
# Nombre d'observations exportées par questionnaire
p_obs_exp, b_obs_exp, i_obs_exp = [x.shape[0] for x in dfs]
# Nombre de variables par questionnaire
p_n_var, b_n_var, i_n_var = [x.shape[1] for x in dfs]

Nombre de variables et d'observations par questionnaire :
- 167 variables et 8719 observations pour le questionnaire Implantation
- 189 variables et 7766 observations pour le questionnaire Bâtiments
- 34 variables et 1083 observations pour le questionnaire PO

In [4]:
# Informations sur les questions
i_info = pd.read_excel(folder+impl_file,
              sheet_name="Informations",
              usecols="C:E",
              skiprows=12
             )
b_info = pd.read_excel(folder+bat_file_old,
              sheet_name="information",
              usecols="C:E",
              skiprows=11
             )
p_info = pd.read_excel(folder+po_file,
              sheet_name="informations",
              usecols="C:E",
              skiprows=12
             )
infos = [i_info, b_info, p_info]
for info in infos:
    info['id_var'] = info["Unnamed: 2"].apply(lambda x:x.split(". ")[0])
    info['Unnamed: 2'] = info['Unnamed: 2'].apply(lambda x:x.split(". ")[1])
    info.columns = ["variable","type_var","choix","id_var"]
    
i_info['quest'], b_info['quest'], p_info['quest'] = 'impl','bat','po'
infos = pd.concat([b_info, i_info, p_info])

Les questions appellent différents types de réponses :
- FU : Fermée Unique
- ON : Ouverte Numérique
- OT : Ouverte Texte
- OC : Ouverte Code
- FE : Fermée Echelle
- FM : Fermée Multiple

A chaque type de réponses correspondra un traitement particulier.

## 5. Sélection des Réseaux d'enseignement

In [5]:
# Le nombre de réseau varie selon le questionnaire : 
i = i_df.reseau.dropna().unique().tolist()
# 7 réseaux dans le questionnaire Implantation
p = p_df.reseau.dropna().unique().tolist() 
# 6 réseaux dans le questionnaire PO
b = b_df.reseau.dropna().unique().tolist() 
# 5 réseaux dans le questionnaire Bâtiment

Deux manipulations sont à opérer ici :
- il est demandé de mettre les résultats de la COCOF dans le fichier de résultats du subventionné provincial (pas possible avant l'échéance du 8 novembre) ;
- sélectionner les réseaux qui sont présents dans les 3 questionnaires

In [37]:
RESEAUX = set(i).intersection(p,b)
RESEAUX

{'Libre confessionnel',
 'Libre non confessionnel',
 'Subventionné communal',
 'Subventionné provincial',
 'WBE'}

## 6. Fonctions utiles

In [15]:
###################
### Mise en page ##
###################
def formatage_titre(feuille):
    return feuille.add_format({
        'font_size':16,
        'font_color':'blue',
        'align':'center',
        'valign':'vcenter',
        'border':2        
    })

def formatage_question(feuille):
    return feuille.add_format({
    'bold':True,
    'bg_color':'navy',
    'font_color':'white'        
    })

def formatage_entete_col(feuille):
    return feuille.add_format({
            'bold':True,
        'align':'center'
    })

def formatage_reponse(feuille):
    return feuille.add_format({
        'align':'right'
    })

def formatage_stats(feuille):
    return feuille.add_format({
        'align':'left',
        'bold':True,
    })
###################
### Traitements ###
###################
def taux_de_reponse(serie_total, obs_total):
    return round((serie_total/obs_total)*100, 2)

def traitement_ON(serie, obs, df, label, bins):
    '''Réponses Ouvertes Numériques'''
    r1 = serie.count()
    t1 = taux_de_reponse(r1, n_obs_i)
    moyenne = round(serie.mean(), 2)
    ecart_type = round(serie.std(), 2)
    min_value = serie.min()
    max_value = serie.max()
    interval = pd.cut(serie, bins=bins, labels=label, duplicates='drop')
    fx = pd.DataFrame(df.groupby(interval).size()).rename(columns={0:'Nb'})
    fx['%obs'] = fx.Nb.apply(lambda x:round((x/r1)*100,2))
    fx.Nb = fx.Nb.astype(int)    
    return fx, t1, moyenne, ecart_type

def traitement_FU(serie, obs, nom_col, df):
    '''Réponses Fermées Uniques'''
    r1 = serie.count()
    tx1 = taux_de_reponse(r1, obs)
    f1 = pd.DataFrame(df.groupby(nom_col).size()).rename(columns={0:'Nb'})
    f1['%obs'] = f1.Nb.apply(lambda x:round((x/r1)*100,2))
    f1.loc['Total'] = round(f1.sum(),0)
    f1.Nb = f1.Nb.astype(int)
    return f1, tx1

def traitement_FM(serie, obs, df):
    '''Réponses Fermées Multiples'''
    r1 = serie.count()
    tx1 = taux_de_reponse(r1, obs)
    f3 = pd.DataFrame(serie.astype(str).str.split(pat=";", expand=True)
                      .stack().value_counts()).rename(columns={0:'Nb'})
    f3 = f3[f3.index != 'nan']
    f3['%obs'] = f3.Nb.apply(lambda x:round((x/r1)*100,2))
    f3.Nb = f3.Nb.astype(int)
    return f3, tx1

def traitement_FE(sqst,col, qst, feuille, frmt1, frmt2, frmt3, df, n_obs):
    '''Réponses Fermées à Echelle'''
    feuille.write(feuille.dim_rowmax + 1,0,qst,frmt1)
    for l in sqst:
        r4 = df[col+l[0]].count()
        tr4 = round((r4/n_obs)*100, 2)
        tx4 = taux_de_reponse(r4, n_obs)
        f4 = pd.DataFrame(df.groupby(col+l[0]).size()).rename(columns={0:'Nb'})
        f4['%obs'] = f4.Nb.apply(lambda x:round((x/r4)*100,2))
        f4.loc['Total'] = round(f4.sum(),0)
        f4.Nb = f4.Nb.astype(int)
        inscription_question(feuille, frmt1, frmt2, tx4, col+l[0]+' : '+l[1])
        inscription_resultats(f4, feuille.dim_rowmax + 1, frmt3, feuille)

################
### Ecriture ###
################
def inscription_statistiques(moyenne, ecarttype, sheet, frmt):
    dbt_stat = sheet.dim_rowmax + 1
    sheet.write(dbt_stat, 0, "Moyenne: "+str(moyenne)+" // Ecart-Type: "+str(ecarttype), frmt)
    
def inscription_resultats(df,ligne,frmt,sheet):
    for x in df.itertuples():
        sheet.write(ligne,col,x[0],frmt)
        sheet.write(ligne,col+1,x[1])
        sheet.write(ligne,col+2,x[2])
        ligne += 1

def inscription_question(sheet, frmt1, frmt2, tdr, question):
    dbt_nv_tbl = sheet.dim_rowmax + 1
    sheet.write(dbt_nv_tbl,0,question,frmt1)
    sheet.write(dbt_nv_tbl+1,1,'Nb',frmt2)
    sheet.write(dbt_nv_tbl+1,2,'%obs',frmt2)
    sheet.write(dbt_nv_tbl+1,0,'Taux de réponse: '+str(tdr)+'%')

## 7. Questions

In [16]:
# Questions PO
q1 = "1. Existe-t-il un responsable ou un service dédié chargé de la gestion des infrastructures ?"
q2 = "2. Au cours des 5 dernières années, vos établissements ont-ils fait l'objet de travaux autres que de maintenance ?"
q3 = "3. Quelles ont été les sources de financement pour ces travaux ?"
q4 = "4. De quel(s) type(s) de financements publics s'agissait-il ?"
q5 = "5. Pourquoi n'avez-vous pas fait appel aux subventions de la FWB ?"
q6 = "6. Dans le cadre du PEE, avez-vous la volonté de faire évoluer vos bâtiments au regard des thématiques suivantes ?"
q7 = "7. Avez-vous déjà réalisé une stratégie de rénovation immobilière énergétique pour votre implantation ?"
q8 = "8. Existe-t-il au sein du PO un SIPPT et/ou un SEPPT"
# Questions IMPLANTATION
q9 = "1. Les locaux de l'implantation sont-ils conformes aux besoins en terme de surface?"
q10 = "2. Existe-t-il des possibilités d'agrandissement à proximité de cette implantation (Non compris pavillon préfabriqués) ?"
q11 = "3. Pour chacun des types de locaux et espaces suivants, pouvez-vous nous dire si cette implantation en dispose ?"
q12 = "4. Si votre chef.fe d'établissement vous a fait part d'une inadéquation à ses besoins?"
q13 = "5. Justifiez l'inadéquation : Ce type local/d'espace est inadéquat car :"
q14 = "5.1 Votre PO est-il propriétaire d’ au moins un bâtiment utilisé par cette implantation ?"
q15 = "5.1.a Par ailleurs, votre PO est-il propriétaire d’un ou plusieurs bâtiments qu’il est sur le point (dans les 12 mois a venir) de vendre, de quitter pour déménagement, autrement dit, des bâtiments dont votre PO ne va plus avoir la gestion, la propriété ?"
q16 = "5.1.b Sous quels statut votre PO utilise-t-il des bâtiments dont il n’est pas propriétaire ?"
q17 = "5.1.c Quelle est la surface brute totale utilisée par cette implantation dans les bâtiments dont vous n’êtes pas propriétaire et/ou que vous êtes sur le point de quitter (déménagement, vente, etc) ?"
q18 = "6. A votre connaissance, les utilisateurs de l'implantation ont-ils recours à des infrastructures en dehors des locaux de celle-ci ?"
q19 = "7. S'agit-il d'infrastructure(s)…"
q20 = "8. Cette implantation partage-t-elle une partie des locaux, des bâtiments avec une autre implantation ?"
q21 = "12. Quelle est la surface totale des cours de récréation de l'implantation ? (En m²)"
q22 = "13. Dans quel état se trouvent globalement les revêtements de celles-ci ?"
q23 = "13.1. Quelle est la surface totale concernée pour un besoin de rénovations ? (% de la surface totale)"
q24 = "14. Positionnez-vous sur l'échelle suivante pour chacune des propositions :"
q25 = "15. Quelles sont les possibilités existantes à proximité directe de l'implantation en termes de modes de déplacements alternatifs à la voiture ?"
q26 = "16. Disposez-vous de suffisamment de :"
q27 = "17. Quelles sont les infrastructures dont l'implantation dispose pour en faciliter l'accès aux personnes ayant un handicap ?"
q28 = "18. Existe-t-il un conseiller en prévention au niveau de l'implantation ?"
q29 = "19. Votre implantation dispose-t-elle d'un accès à l'Internet ?"
q30 = "20. Le débit internet est-il suffisant au regard de vos besoins ?"
q31 = "21. Au sein de votre implantation, disposez-vous d'un réseau de câblages électrique et informatique interne permettant une utilisation des outils numériques à des fins pédagogiques ?"
q32 = "22. Celui-ci est-il réparti sur l'ensemble des bâtiments de l'implantation ?"
q33 = "23. L'implantation génère-t-elle de l'énergie elle-même via…"
q34 = "24. Y a-t-il dans l'implantation un système de récupération d'eau de pluie ?"
q35 = "25. Celui-ci est-il suffisant pour couvrir les besoins en eau non potable de l'implantation ?"
q36 = "26. Les sanitaires sont-ils"
q37 = "27. Y a-t-il des problèmes dans ces sanitaires ?"
q38 = "28. Y a-t-il des éviers en suffisance…"
q39 = "29. Afin de compléter les parties du questionnaire concernant l'usage des bâtiments, avez-vous fait appel à la/aux direction(s) des établissements dont vous avez la gestion ?"
# Questions BATIMENTS
q40 = "1. A quel type de construction appartient ce bâtiment ?"
q41 = "2. Ce bâtiment a-t-il à l’origine été construit pour un usage scolaire ?"
q42 = "3. De quand date le bâtiment ?"
q43 = "4. Depuis sa construction, le bâtiment a-t-il connu une rénovation lourde ?"
q44 = "4.1. précisez l'année de la dernière rénovation lourde"
q45 = "5. Quel(le) est en m² la surface bâtie au sol (surface brute : emprise au sol)"
q46 = "6. Quelles sont les destinations principales du bâtiment ?"
q47 = "7. Une partie du bâtiment est-elle actuellement inoccupée ?"
q48 = "7.1. Précisez la surface concernée (% de la surface brute du bâtiment)"
q49 = "8. Précisez pour quelle raison le bâtiment est inoccupé (en tout ou en partie)"
q50 = "8.1. En raison d'un risque pour les utilisateurs ? Le(s)quel(s) ?"
q51 = "8.2. En raison d'un problème majeur de praticabilité ? Le(s)quel(s) ?"
q52 = "9. Les travaux nécessaires à la réouverture sont-ils programmés ?"
q53 = "10. Si oui, quand ?"
q54 = "11. Disposez-vous d'une estimation chiffrée des besoins en vue de la réouverture ?"
q55 = "11.1 Précisez l'estimation (en euros)"
q56 = "12. Pour chacun des types de locaux suivants, pouvez-vous nous dire si votre implantation en dispose ?"
q57 = "13. Votre chef.fe d'établissement vous a fait part d'une inadéquation à ses besoins (soit des locaux en nombre insuffisants et/ou de dimensions insuffisantes) ?"
q58 = "14. Justifiez l'inadéquation : ce type de local/d'espace est inadéquat car :"
q59 ="15. Au regard des rapports listés ci-dessous, un problème de conformité aux normes légales a-t-il été constaté dans le bâtiment ?"
q60 = "16. Uniquement pour le supérieur et si Q15 = 3 ou 4, disposez-vous d'une estimation chiffrée pour la mise aux normes ?"
q61 = "17. Dans quel état se trouvent...."
q62 = "17.1 Dossier travaux en cours"
q63 = "18. Disposez-vous d'une comptabilité énergétique ?"
q64 = "19. Quelle est la consommation annuelle ?"
q65 = "20. Quel(s) est/sont le(s) type(s) de combustible utilisé(s) pour chauffer le bâtiment ?"
q66 = "21. De quelle année date l'unité de production (chaudière ou autre) ?"
q67 = "22. De quelle année date l'installation de chauffage ?"
q68 = "23. Les installations sont-elles couvertes par un contrat d'entretien ?"
q69 = "24. Le bâtiment possède-t-il une ventilation mécanique ?"
q70 = "25. La ventilation est-elle suffisante en fonction de la norme ?"
q71 = "25.a Si non, quelle est la valeur … en m³/h/pers"
q72 = "26. Le bâtiment dispose-t-il d'un détecteur de CO2 (qualité de l'air ambiant) ?"
q73 = "27. Le bâtiment possède-t-il une climatisation (production de froid ? ventilation) ?"
q74 = "28. Le bâtiment souffre-t-il de surchauffe due à l'ensoleillement ?"
q75 = "29.a. Disposez-vous d'un certificat PEB bâtiment public ou d'un audit énergétique ?"
q76 = "29.b Précisez la performance énergétique du bâtiment / la consommation annuelle normalisée en énergie primaire :"
q77 = "29.c Disposez-vous d'un audit énergétique ?"
q78 = "30. Quelles sont les mesures d'efficacité énergétique prises pour:"
q79 = "31. Le bâtiment présente-t-il un risque majeur de devoir être fermé à terme (1 an) ?"
q80 = "32. Précisez pour quelle raison"
q81 = "33. Les travaux nécessaires à la réouverture sont-ils programmés ?"
q82 = "34. Si des travaux sont nécessaires..."
q83 = "35. Disposez-vous d'une estimation chiffrée des besoins en vue de la réouverture ?"
q84 = "36. Au cours des 3 dernières années, avez-vous eu des retours de vos utilisateurs concernant : "

In [17]:
# Sélection des répondants, pas pour le questionnaire Bâtiment
p_p = p_df[p_df.Participation == "PO participant"]
i_p = i_df[i_df.Participation == "Participant"]

In [19]:
n_total = len(p_df)
n_nonp = len(p_df[p_df.Participation == "PO non participant"])
n_total, n_nonp

(1083, 505)

## 8. Traitements

In [59]:
for reseau in RESEAUX:
    # Création des fichiers excel, un par réseau
    cahier = pd.ExcelWriter(rapport+'\\'+"Reseau_"+reseau+".xlsx",
                    engine="xlsxwriter",
                    engine_kwargs={'options':{'nan_inf_to_errors': True}})
    wb = cahier.book
    # Création des onglets, un par questionnaire
    f_po = wb.add_worksheet('Questionnaire PO')
    f_imp = wb.add_worksheet('Questionnaire IMPLANTATION')
    f_bat = wb.add_worksheet('Questionnaire BATIMENT')
    # Mise en page globale
    frmt_titre = formatage_titre(wb)
    frmt_quest = formatage_question(wb)
    frmt_entete_colonne = formatage_entete_col(wb)
    frmt_rep = formatage_reponse(wb)
    frmt_stat = formatage_stats(wb)
####################################
### Traitements QUESTIONNAIRE PO ###
####################################
    # sélection par réseau
    if not reseau == "Subventionné provincial":
        df = p_p[p_p['reseau'] == reseau]
    else:
        df = p_p.loc[(p_p['reseau'] == reseau) | (p_p['reseau'] == "COCOF")]

###### Participation des PO selon le réseau d’enseignement (% Pop totale)
    n_total = len(p_df[p_df.reseau == reseau])
    # Nombre de répondants par réseau
    n_obs = len(df)
    # Nombre de non-répondants par réseau
    n_nonp = p_df[(p_df.Participation == "PO non participant") & (p_df.reseau == reseau)]
    # Taux de participation
    tx_part = round((n_obs/n_total)*100, 2)
    # Mise en page
    f_po.set_column('A1:A1',len(q6))
    f_po.merge_range('A1:C1',"Tableau de bord PO : Reseau "+reseau, frmt_titre)
    f_po.merge_range('A2:C2',"Nombre de PO participants : "+str(n_obs)+" || Nombre de PO dans le réseau "+reseau+" : "+str(n_total)+" || Taux de répondants : "+str(tx_part)+"%")
    f_po.set_row(0,25)
    col = 0
###### Question n°1 ######
    f1, tx1 = traitement_FU(df.Q01, n_obs, 'Q01', df)
    inscription_question(f_po, frmt_quest, frmt_entete_colonne, tx1, q1)
    inscription_resultats(f1, f_po.dim_rowmax + 1, frmt_rep, f_po)
###### Question n°02 ######
    f2, tx2 = traitement_FU(df.Q02, n_obs, 'Q02', df)
    inscription_question(f_po, frmt_quest, frmt_entete_colonne, tx2, q2)
    inscription_resultats(f2, f_po.dim_rowmax + 1, frmt_rep, f_po)
###### Question n°03 ######
    f3, tx3 = traitement_FM(df.Q03, n_obs, df)
    inscription_question(f_po, frmt_quest, frmt_entete_colonne, tx3, q3)
    inscription_resultats(f3, f_po.dim_rowmax + 1, frmt_rep, f_po)
###### Question n°04 ######
    f4, tx4 = traitement_FM(df.Q04, n_obs, df)
    inscription_question(f_po, frmt_quest, frmt_entete_colonne, tx4, q4)
    inscription_resultats(f4, f_po.dim_rowmax + 1, frmt_rep, f_po)
###### Question n°05 ######
    f5, tx5 = traitement_FM(df.Q05, n_obs, df)
    inscription_question(f_po, frmt_quest, frmt_entete_colonne, tx5, q5)
    inscription_resultats(f5, f_po.dim_rowmax + 1, frmt_rep, f_po)
###### Question n°06 ######
    sq6 = [('a','Tronc commun'),
          ('b','Numérique'),
          ('c','Volet polytechnique'),
          ('d','Végétalisation des espace'),
          ('e','Mutualisation des équipements'),
          ('f','Inclusion'),
          ('g','Ouverture sur le quartier'),
          ('h','Autre')]
    traitement_FE(sq6, 'Q06_', q6, f_po, frmt_quest, frmt_entete_colonne, frmt_rep, df, n_obs)
###### Question n°07 ######
    f7, tx7 = traitement_FU(df.Q07, n_obs, 'Q07', df)
    inscription_question(f_po, frmt_quest, frmt_entete_colonne, tx7, q7)
    inscription_resultats(f7, f_po.dim_rowmax + 1, frmt_rep, f_po)
###### Question n°08 ######
    sq8 = [('a','...un SIPPT'),
          ('b','...un SEPPT')]
    traitement_FE(sq8,'Q08_',q8,f_po,frmt_quest, frmt_entete_colonne, frmt_rep, df, n_obs)
##############################################
### Traitements QUESTIONNAIRE IMPLANTATION ###
##############################################
    # sélection par réseau
    if not reseau == "Subventionné provincial":
        dfi = i_p[i_p['reseau'] == reseau]
    else:
        dfi = i_p.loc[(i_p['reseau'] == reseau) | (i_p['reseau'] == "COCOF")]
# Participation des implantations selon le réseau d’enseignement (% Pop totale)
    n_total_i = len(i_df[i_df.reseau == reseau])
    # Nombre de répondants par réseau
    n_obs_i = len(dfi)
    # Nombre de non-répondants par réseau
    n_nonp_i = i_df[(i_df.Participation == "Participant") & (i_df.reseau == reseau)]
    # Taux de participation
    tx_part_i = round((n_obs_i/n_total_i)*100, 2)
    # Mise en page
    f_imp.set_column('A1:A1',len(q9))
    f_imp.merge_range('A1:C1',"Tableau de bord Implantations : Réseau "+reseau, frmt_titre)
    f_imp.merge_range('A2:C2',"Nombre d'Implantations participantes : "+str(n_obs_i))
    f_imp.merge_range('A3:C3', "Nombre d'Implantations dans le réseau "+reseau+" : "+str(n_total_i)+" || Taux de répondants : "+str(tx_part_i)+"%")
    f_imp.set_row(0,35)
    col = 0
###### Question n°09 ######
    f9, tx9 = traitement_FU(dfi.Q01, n_obs_i, 'Q01', dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx9, q9)
    inscription_resultats(f9, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°10 ######
    f10, tx10 = traitement_FM(dfi.Q02, n_obs_i, dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx10, q10)
    inscription_resultats(f10, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°11 ######
    sq11 = [("a","Locaux pédagogiques"),
            ("b","Locaux administratifs"),
            ("c","Locaux d'éducation physique"),
            ("d","Logement – chambres individuelles ou collectives (dortoir)"),
            ("e","Locaux de restauration, réfectoire"),
            ("f","Cuisine"),
            ("g","Locaux de détente et d'étude (salle d'étude, bibliothèque, salle informatique, salle TV, foyer)"),
            ("h","Locaux consultation médicale (salle d'attente, cabine déshabillage, cabinet de consultation médicale)"),
            ("i","Salle(s) polyvalente(s)"),
            ("j","Espaces de stockages"),
            ("k","Espace(s) bibliothèque(s)"),
            ("l","Laboratoires et locaux spécialisés (pour activités artistiques et technologiques)"),
            ("m","Ateliers de pratiques professionnelles"),
            ("n","Cour(s) de récréation"),
            ("o","Préau(x)"),
            ("p","Terrain(s) de sport praticable"),
            ("q","Espaces verts"),
            ("r","Sanitaires (y compris douches)")]
    traitement_FE(sq11,'Q03_',q11,f_imp, frmt_quest, frmt_entete_colonne, frmt_rep, dfi, n_obs_i)
###### Question n°12 ######
    traitement_FE(sq11,'Q04_',q12,f_imp,frmt_quest, frmt_entete_colonne, frmt_rep, dfi, n_obs_i)
###### Question n°13 ######
    traitement_FE(sq11,'Q05_',q13,f_imp,frmt_quest, frmt_entete_colonne, frmt_rep, dfi, n_obs_i)
###### Question n°14 ######
    f14, tx14 = traitement_FU(dfi.Q05_1, n_obs_i, 'Q05_1', dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx14, q14)
    inscription_resultats(f14, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°15 ######
    f15, tx15 = traitement_FU(dfi.Q05_1_a, n_obs_i, 'Q05_1_a', dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx15, q15)
    inscription_resultats(f15, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°16 ######
    f16, tx16 = traitement_FU(dfi.Q05_1_b, n_obs_i, 'Q05_1_b', dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx16, q16)
    inscription_resultats(f16, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°17 ######
    l_17 = ['Moins de 100m²',
              'Entre 101 et 500m²',
              'Entre 501 et 1.000m²',
              'Entre 1.001 et 2.500m²',
              'Entre 2.501 et 5.000m²',
              'Entre 5.001 et 10.000m²',
              'Entre 10.001 et 20.000m²',
              'Entre 20.001 et 30.000m²',
              'Entre 30.001 et 50.000m²']
    b_17 = (0, 100, 500, 1000, 2500, 5000, 10000, 20000, 30000, 50000)
    f17, tx17, moyenne, ecart_type = traitement_ON(dfi.Q05_1_c, n_obs_i, dfi, l_17, b_17)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx17, q17)
    inscription_statistiques(moyenne, ecart_type, f_imp, frmt_stat)
    inscription_resultats(f17, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°18 ######
    f18, tx18 = traitement_FU(dfi.Q06, n_obs_i, 'Q06', dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx18, q18)
    inscription_resultats(f18, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°19 ######
    f19, tx19 = traitement_FM(dfi.Q07, n_obs_i, dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx19, q19)
    inscription_resultats(f19, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°20 ######
    f20, tx20 = traitement_FU(dfi.Q08, n_obs_i, 'Q08', dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx20, q20)
    inscription_resultats(f20, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°21 ######
    f21, tx21, m21, ect21 = traitement_ON(dfi.Q12, n_obs_i, dfi, l_17, b_17)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx21, q21)
    inscription_statistiques(m21, ect21, f_imp, frmt_stat)
    inscription_resultats(f21, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°22 ######
    f22, tx22 = traitement_FU(dfi.Q13, n_obs_i, 'Q13', dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx22, q22)
    inscription_resultats(f22, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°23 ######
    l_23 = ['Moins de 10%',
              'Entre 10 et 25%',
              'Entre 26 et 50%',
              'Entre 51 et 75%',
              'Entre 76 et 100%']
    b_23 = (0, 10, 25, 50, 75, 100)
    f23, tx23, m23, ect23 = traitement_ON(dfi.Q13_1, n_obs_i, dfi, l_23, b_23)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx23, q23)
    inscription_statistiques(m23, ect23, f_imp, frmt_stat)
    inscription_resultats(f23, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°24 ######
    sq24 = [
        ('a','La fonctionnalité est généralement bonne'),
        ('b',"Les bâtiments offrent un bon soutien au projet pédagogique de l'établissement"),
        ('c',"Les locaux sont suffisamment flexibles et multifonctionnels à utiliser"),
        ("d","L'implantation est un lieu accueillant où les utilisateurs peuvent se sentir bien et sans discrimination"),
        ("e","La cour de récréation permet une diversité d'ambiances et la cohabitation harmonieuse des usages"),
        ("f","Dans la cours de récréation, il y a un bon équilibre entre les espaces végétalisés et les espaces minéralisés")
    ]
    traitement_FE(sq24,'Q14_',q24,f_imp,frmt_quest, frmt_entete_colonne, frmt_rep, dfi, n_obs_i)
###### Question n°25 ######
    f25, tx25 = traitement_FM(dfi.Q15, n_obs_i, dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx25, q25)
    inscription_resultats(f25, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°26 ######
    sq26 = [
        ("a","Places de parking pour le personnel"),
        ("b","Places de parking pour les étudiants"),
        ("c","Parkings vélos/motos")
    ]
    traitement_FE(sq26,'Q16_',q26,f_imp,frmt_quest, frmt_entete_colonne, frmt_rep, dfi, n_obs_i)
###### Question n°27 ######
    sq27 = [
        ("a","Place(s) de parking adapté PMR à proximité de l'entrée"),
        ("b","Accessibilité aux étages (ascenseur adapté, rampe, etc.)"),
        ("c","Sanitaires adaptés"),
        ("d","Autre")
    ]
    traitement_FE(sq27,'Q17_',q27,f_imp,frmt_quest, frmt_entete_colonne, frmt_rep, dfi, n_obs_i)
###### Question n°28 ######
    f28, tx28 = traitement_FU(dfi.Q18, n_obs_i, 'Q18', dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx28, q28)
    inscription_resultats(f28, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°29 ######
    f29, tx29 = traitement_FM(dfi.Q19, n_obs_i, dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx29, q29)
    inscription_resultats(f29, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°30 ######
    f30, tx30 = traitement_FU(dfi.Q20, n_obs_i, 'Q20', dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx30, q30)
    inscription_resultats(f30, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°31 ######
    f31, tx31 = traitement_FU(dfi.Q21, n_obs_i, 'Q21', dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx31, q31)
    inscription_resultats(f31, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°32 ######
    f32, tx32 = traitement_FU(dfi.Q22, n_obs_i, 'Q22', dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx32, q32)
    inscription_resultats(f32, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°33 ######
    f33, tx33 = traitement_FM(dfi.Q23, n_obs_i, dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx33, q33)
    inscription_resultats(f33, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°34 ######
    f34, tx34 = traitement_FU(dfi.Q24, n_obs_i, 'Q24', dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx34, q34)
    inscription_resultats(f34, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°35 ######
    f35, tx35 = traitement_FU(dfi.Q25, n_obs_i, 'Q25', dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx35, q35)
    inscription_resultats(f35, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Question n°36 ######
    sq36 = [
        ("a","En nombre suffisant"),
        ("b","En état"),
        ("c","Adapté à l'âge des différents utilisateurs"),
        ("d","Unisexes"),
        ("e","Distincts pour le personnel")]
    traitement_FE(sq36,'Q26_',q36,f_imp,frmt_quest, frmt_entete_colonne, frmt_rep, dfi, n_obs_i)
###### Question n°37 ######
    sq37 = [
        ("a","Des problèmes de ventilation"),
        ("b","Des problèmes d'évacuation des eaux usées")]
    traitement_FE(sq37,'Q27_',q37,f_imp,frmt_quest, frmt_entete_colonne, frmt_rep, dfi, n_obs_i)
###### Question n°38 ######
    sq38 = [
        ("a","Dans les sanitaires"),
        ("b","Dans les classes")]
    traitement_FE(sq38,'Q28_',q38,f_imp,frmt_quest, frmt_entete_colonne, frmt_rep, dfi, n_obs_i)
###### Question n°39 ######
    f39, tx39 = traitement_FU(dfi.Q29, n_obs_i, 'Q29', dfi)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, tx39, q39)
    inscription_resultats(f39, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###### Nombre de bâtiments ######
    l_bat = ['Entre 1 et 5',
              'Entre 6 et 10',
             'Entre 11 et 25',
             'Plus de 25']
    b_bat = (0, 5, 10, 25, 4000)
    fbat, txbat, mbat, ectbat = traitement_ON(dfi.Nombre_Batiments, n_obs_i, dfi, l_bat, b_bat)
    inscription_question(f_imp, frmt_quest, frmt_entete_colonne, txbat, 'Nombre de bâtiments')
    inscription_statistiques(mbat, ectbat, f_imp, frmt_stat)
    inscription_resultats(fbat, f_imp.dim_rowmax + 1, frmt_rep, f_imp)
###########################################
### Traitements QUESTIONNAIRE BATIMENTS ###
###########################################
    dfb = b_df[b_df.reseau == reseau]
    n_obs_b = len(dfb)
    # Mise en page
    f_bat.set_column('A1:A1',len(q9))
    f_bat.merge_range('A1:C1',"Tableau de bord Bâtiments : Réseau "+reseau, frmt_titre)
    f_bat.merge_range('A2:C2',"Nombre d'observations : "+str(n_obs_b))
    f_bat.set_row(0,25)
    col = 0
######### Nature ##########
    fn, txn = traitement_FU(dfb.nature, n_obs_b, 'nature', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, txn, 'Nature')
    inscription_resultats(fn, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°40 ######
    f40, tx40 = traitement_FU(dfb.Q01, n_obs_b, 'Q01', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx40, q40)
    inscription_resultats(f40, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°41 ######
    f41, tx41 = traitement_FU(dfb.Q02, n_obs_b, 'Q02', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx41, q41)
    inscription_resultats(f41, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°42 ######
    f42, tx42 = traitement_FU(dfb.Q03, n_obs_b, 'Q03', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx42, q42)
    inscription_resultats(f42, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°43 ######
    f43, tx43 = traitement_FU(dfb.Q04, n_obs_b, 'Q04', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx43, q43)
    inscription_resultats(f43, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°44 ######
    l_44 = ['Avant 1975',
            'Entre 1976 et 2000',
            'Entre 2001 et 2010',
            'Après 2011']
    b_44 = (0, 1975, 2000, 2010, 2050)
    f44, tx44, m44, ect44 = traitement_ON(dfb.Q04_1, n_obs_b, dfb, l_44, b_44)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx44, q44)
    inscription_resultats(f44, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°45 ######
    l_45 = ['Moins de 500 m²',
            'Entre 501 et 1000 m²',
            'Entre 1001 et 2500 m²',
           'Plus de 2500']
    b_45 = (0, 500, 1000, 2500, 100000)
    f45, tx45, m45, ect45 = traitement_ON(dfb.Q05_a, n_obs_b, dfb, l_45, b_45)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx45, q45)
    inscription_statistiques(m45, ect45, f_bat, frmt_stat)
    inscription_resultats(f45, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°46 ######
    f46_a = pd.DataFrame(dfb.groupby('Q06_a').size()).rename(columns={0:'Nb'}).reset_index().rename({'Q06_a':'destination'}, axis='columns')
    f46_b = pd.DataFrame(dfb.groupby('Q06_b').size()).rename(columns={0:'Nb'}).reset_index().rename({'Q06_b':'destination'}, axis='columns')
    f46_c = pd.DataFrame(dfb.groupby('Q06_c').size()).rename(columns={0:'Nb'}).reset_index().rename({'Q06_c':'destination'}, axis='columns')
    f_a_concat = pd.concat([f46_a, f46_b, f46_c])
    f46 = pd.DataFrame(f_a_concat.groupby('destination')['Nb'].sum())
    f46['%obs'] = f46.Nb.apply(lambda x:round((x/f46.Nb.sum())*100,2))
    # f46.loc['Total'] = round(f46.Nb.sum(),0)
    f46.Nb = f46.Nb.astype(int)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, "Tableau remanié", q46)
    inscription_resultats(f46, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°47 ######
    f47, tx47 = traitement_FU(dfb.Q07, n_obs_b, 'Q07', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx47, q47)
    inscription_resultats(f47, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°48 ######
    f48, tx48, m48, ect48 = traitement_ON(dfb.Q07_1, n_obs_b, dfb, l_23, b_23)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx48, q48)
    inscription_statistiques(m48, ect48, f_bat, frmt_stat)
    inscription_resultats(f48, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°49 ######
# "8. Précisez pour quelle raison le bâtiment est inoccupé (en tout ou en partie)"
    f49, tx49 = traitement_FU(dfb.Q08, n_obs_b, 'Q08', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx49, q49)
    inscription_resultats(f49, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°50 ######
# "8.1. En raison d'un risque pour les utilisateurs ? Le(s)quel(s) ?"
    series = ['Q08_1_'+str(x) for x in ['a','b','d','e','f','g','h']]
    f_concat = pd.concat([pd.DataFrame(dfb.groupby(serie).size())
            .rename(columns={0:'Nb'})
            .reset_index()
            .rename({serie:'risque'}, axis='columns') for serie in series])
    f50 = pd.DataFrame(f_concat.groupby('risque')['Nb'].sum())
    f50['%obs'] = f50.Nb.apply(lambda x:round((x/f50.Nb.sum())*100,2))
    f50.Nb = f50.Nb.astype(int)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, "Tableau remanié", q50)
    inscription_resultats(f50, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°51 ######
# "8.2. En raison d'un problème majeur de praticabilité ? Le(s)quel(s) ?"
    s51 = ['Q08_2_'+str(x) for x in ['a','b','d','e','f']]
    f_concat = pd.concat([pd.DataFrame(dfb.groupby(s).size())
            .rename(columns={0:'Nb'})
            .reset_index()
            .rename({s:'problème'}, axis='columns') for s in s51])
    f51 = pd.DataFrame(f_concat.groupby('problème')['Nb'].sum())
    f51['%obs'] = f51.Nb.apply(lambda x:round((x/f51.Nb.sum())*100,2))
    f51.Nb = f51.Nb.astype(int)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, "Tableau remanié", q51)
    inscription_resultats(f51, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°52 ######
# "9. Les travaux nécessaires à la réouverture sont-ils programmés ?"
    f52, tx52 = traitement_FU(dfb.Q09, n_obs_b, 'Q09', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx52, q52)
    inscription_resultats(f52, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°53 ######
# "10. Si oui, quand ?"
    f53, tx53 = traitement_FU(dfb.Q10, n_obs_b, 'Q10', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx53, q53)
    inscription_resultats(f53, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°54 ######
# "11. Disposez-vous d'une estimation chiffrée des besoins en vue de la réouverture ?"
    f54, tx54 = traitement_FU(dfb.Q11, n_obs_b, 'Q11', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx53, q54)
    inscription_resultats(f54, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°55 ######
# "11.1 Précisez l'estimation (en euros)"
# Si peu de réponses qu'on va tous les mettre, plutôt que d'en faire des classes
    f55, tx55 = traitement_FU(dfb.Q11_1, n_obs_b, 'Q11_1', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx53, q55)
    inscription_resultats(f55, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°56 ######
# "12. Pour chacun des types de locaux suivants, pouvez-vous nous dire si votre implantation en dispose ?"
    sq56 = [
        ("a", "Locaux dédiés à l'enseignement théorique"),
        ("b", "Locaux dédiés à la recherche"),
        ("c", "Locaux dédiés à l'enseignement pratique"),
        ("d", "Locaux administratifs"),
        ("e", "Des espaces de stockage"),
        ("f", "Espaces de socialisation/lieux de vie"),
        ("g", "« Learning center » (ou assimilé)"),
        ("h", "Espace(s) bibliothèque(s)")
    ]
    traitement_FE(sq56,'Q12_',q56,f_bat,frmt_quest, frmt_entete_colonne, frmt_rep, dfb, n_obs_b)
###### Question n°57 ######
# "13. Votre chef.fe d'établissement vous a fait part d'une inadéquation à ses besoins 
# (soit des locaux en nombre insuffisants et/ou de dimensions insuffisantes) ?"
    traitement_FE(sq56,'Q13_',q57,f_bat,frmt_quest, frmt_entete_colonne, frmt_rep, dfb, n_obs_b)
###### Question n°58 ######
# "14. Justifiez l'inadéquation : ce type de local/d'espace est inadéquat car :"
    traitement_FE(sq56,'Q14_',q58,f_bat,frmt_quest, frmt_entete_colonne, frmt_rep, dfb, n_obs_b)
###### Question n°59 ######
# "15. Au regard des rapports listés ci-dessous, 
# un problème de conformité aux normes légales a-t-il été constaté dans le bâtiment ?"
# A quoi sert la variable Q015_g ?
    sq59 = [
        ("a", "Rapport des services de pompier"),
        ("b", "Rapport inspection amiante"),
        ("c", "Rapport de conformité électrique"),
        ("d", "Rapport SPF Emploi (conditions de travail)"),
        ("e", "Permis d'environnement"),
        ("f", "Etude de stabilité")]
    traitement_FE(sq59,'Q15_',q59,f_bat,frmt_quest, frmt_entete_colonne, frmt_rep, dfb, n_obs_b)
###### Question n°60 ######
# "16. Uniquement pour le supérieur et si Q15 = 3 ou 4, 
# disposez-vous d'une estimation chiffrée pour la mise aux normes ?"
    traitement_FE(sq59,'Q16_',q60,f_bat,frmt_quest, frmt_entete_colonne, frmt_rep, dfb, n_obs_b)
###### Question n°61 ######
# "17. Dans quel état se trouvent...."
    sq61 = [
        ("a", "Les toitures"),
        ("b", "Les façades et murs extérieurs"),
        ("c", "Les menuiseries extérieures"),
        ("d", "Les revêtements des sols, plafonds et murs intérieurs"),
        ("e", "Menuiseries intérieures")]
    traitement_FE(sq61,'Q17_',q61,f_bat,frmt_quest, frmt_entete_colonne, frmt_rep, dfb, n_obs_b)
###### Question n°62 ######
# "17.1 Dossier travaux en cours"
    traitement_FE(sq61,'Q17_1_',q62,f_bat,frmt_quest, frmt_entete_colonne, frmt_rep, dfb, n_obs_b)
###### Question n°63 ######
# "18. Disposez-vous d'une comptabilité énergétique ?"
    f63, tx63 = traitement_FU(dfb.Q18, n_obs_b, 'Q18', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx63, q63)
    inscription_resultats(f63, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°64 ######
# "19. Quelle est la consommation annuelle ?"
    sq64 = [
        ("a","Chauffage du bâtiment par m² de plancher brut ? (kW)"),
        ("b","Electricité du bâtiment par m² de plancher brut ? (kWh)"),
        ("c","Eau ? (m³)")]
    s64 = ['Q19_'+str(x[0]) for x in sq64]
    som64 = [dfb[serie].sum() for serie in s64]
    tx64 = [taux_de_reponse(dfb[x].count(),n_obs_b) for x in s64]
    lab64 = [str(x[1]) for x in sq64]
    f64 = pd.DataFrame({
        'label':lab64,
        'somme':som64,
        'taux de réponse':tx64
    })
# fonction inscription_question remaniée    
    dbt_nv_tbl = f_bat.dim_rowmax + 1
    f_bat.write(dbt_nv_tbl, 0, q64, frmt_quest)
    f_bat.write(dbt_nv_tbl+1,1,'Somme', frmt_entete_colonne)
    f_bat.write(dbt_nv_tbl+1,2,'Taux de réponse', frmt_entete_colonne)
# fonction inscription_resultas remaniée
    ligne = f_bat.dim_rowmax+1
    for x in f64.itertuples():
        f_bat.write(ligne,0,x[1],frmt_rep)
        f_bat.write(ligne,1,x[2])
        f_bat.write(ligne,2,x[3])
        ligne += 1    
###### Question n°65 ######
# "20. Quel(s) est/sont le(s) type(s) de combustible utilisé(s) 
# pour chauffer le bâtiment ?"
    f65, tx65 = traitement_FU(dfb.Q20, n_obs_b, 'Q20', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx65, q65)
    inscription_resultats(f65, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°66 ######
# "21. De quelle année date l'unité de production (chaudière ou autre) ?"
    f66, tx66, m66, ect66 = traitement_ON(dfb.Q21, n_obs_b, dfb, l_44, b_44)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx66, q66)
    inscription_resultats(f66, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°67 ######
# "22. De quelle année date l'installation de chauffage ?"
    f67, tx67, m67, ect67 = traitement_ON(dfb.Q22, n_obs_b, dfb, l_44, b_44)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx67, q67)
    inscription_resultats(f67, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°68 ######
# "23. Les installations sont-elles couvertes 
# par un contrat d'entretien ?"
    f68, tx68 = traitement_FU(dfb.Q23, n_obs_b, 'Q23', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx68, q68)
    inscription_resultats(f68, f_bat.dim_rowmax + 1, frmt_rep, f_bat)    
###### Question n°69 ######
# "24. Le bâtiment possède-t-il une ventilation mécanique ?"
    f69, tx69 = traitement_FU(dfb.Q24, n_obs_b, 'Q24', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx69, q69)
    inscription_resultats(f69, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°70 ######
# "25. La ventilation est-elle suffisante en fonction de la norme ?"
    f70, tx70 = traitement_FU(dfb.Q25, n_obs_b, 'Q25', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx70, q70)
    inscription_resultats(f70, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°71 ######
# "25.a Si non, quelle est la valeur … en m³/h/pers"
    l_bat = ['Entre 1 et 5',
              'Entre 6 et 10',
             'Entre 11 et 25',
             'Plus de 25']
    b_bat = (0, 5, 10, 25, 4000)
    f25a, tx25a, m25a, ect25a = traitement_ON(dfb.Q25_a, n_obs_b, dfb, l_bat, b_bat)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx25a, q71)
    inscription_statistiques(m25a, ect25a, f_bat, frmt_stat)
    inscription_resultats(f25a, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°72 ######
# "26. Le bâtiment dispose-t-il 
# d'un détecteur de CO2 (qualité de l'air ambiant) ?"
    f72, tx72 = traitement_FU(dfb.Q26, n_obs_b, 'Q26', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx72, q72)
    inscription_resultats(f72, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°73 ######
    f73, tx73 = traitement_FU(dfb.Q27, n_obs_b, 'Q27', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx73, q73)
    inscription_resultats(f73, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°74 ######
    f74, tx74 = traitement_FU(dfb.Q28, n_obs_b, 'Q28', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx74, q74)
    inscription_resultats(f74, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°75 ######
# "29.a. Disposez-vous d'un 
# certificat PEB bâtiment public ou d'un audit énergétique ?"
    f75, tx75 = traitement_FU(dfb.Q29_a, n_obs_b, 'Q29_a', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx75, q75)
    inscription_resultats(f75, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°76 ######
# "29.b Précisez la performance énergétique du bâtiment / 
# la consommation annuelle normalisée en énergie primaire :"
    f76, tx76, m76, ect76 = traitement_ON(dfb.Q29_b, n_obs_b, dfb, l_bat, b_bat)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx76, q76)
    inscription_statistiques(m76, ect76, f_bat, frmt_stat)  
###### Question n°77 ######
# "29.c Disposez-vous d'un audit énergétique ?"
    f77, tx77 = traitement_FU(dfb.Q29_c, n_obs_b, 'Q29_c', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx77, q77)
    inscription_resultats(f77, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°78 ######
# "30. Quelles sont les mesures d'efficacité énergétique prises pour:"
    sq78 = [
        ("a", "Isolation des murs"),
        ("b", "Isolation de la toiture"),
        ("c", "Isolation du plancher des combles ou au-dessus de locaux non-chauffés"),
        ("d", "Double vitrage isolant"),
        ("e", "Châssis à coupure thermique"),
        ("f", "Chaudière haut rendement"),
        ("g", "Isolations des conduites et des vannes traversant les locaux non chauffés (chaufferie, gaines techniques, faux-plafonds, …)"),
        ("h", "Système de régulation de chauffage adapté aux besoins selon les horaires, les périodes"),
        ("i", "Vannes thermostatiques positionnées selon l'utilisation des locaux"),
        ("j", "Présence d'une sonde extérieure"),
        ("k", "Protections contre les surchauffes (physique ou mécanique)"),
        ("l", "Lampage LED"),
        ("m", "Minuteries dans les locaux occupés de façon intermittente"),
        ("n", "Contrôle de la lumière du jour")       
    ]
    traitement_FE(sq78,'Q30_',q30,f_bat,frmt_quest, frmt_entete_colonne, frmt_rep, dfb, n_obs_b)
###### Question n°79 ######
    f79, tx79 = traitement_FU(dfb.Q31, n_obs_b, 'Q31', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx79, q79)
    inscription_resultats(f79, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°80 ######
    f80, tx80 = traitement_FU(dfb.Q32, n_obs_b, 'Q32', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx80, q80)
    inscription_resultats(f80, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°81 ######
    f81, tx81 = traitement_FU(dfb.Q33, n_obs_b, 'Q33', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx81, q81)
    inscription_resultats(f81, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°82 ######
    f82, tx82 = traitement_FU(dfb.Q34, n_obs_b, 'Q34', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx82, q82)
    inscription_resultats(f82, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°83 ######
    f83, tx83 = traitement_FU(dfb.Q35, n_obs_b, 'Q35', dfb)
    inscription_question(f_bat, frmt_quest, frmt_entete_colonne, tx83, q83)
    inscription_resultats(f83, f_bat.dim_rowmax + 1, frmt_rep, f_bat)
###### Question n°84 ######
    sq84 = [
        ("a", "Des problèmes d'acoustique"),
        ("b", "Des problèmes de température dans les locaux"),
        ("c", "Des problèmes de manque de luminosité"),
        ("d", "Des problèmes de ventilation"),
        ("e", "Autre")]
    traitement_FE(sq84,'Q36_',q30,f_bat,frmt_quest, frmt_entete_colonne, frmt_rep, dfb, n_obs_b)
###########################
### FIN DES TRAITEMENTS ###
###########################
    cahier.save()